In [22]:
# This is Proj1

In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

24/08/15 16:24:03 WARN Utils: Your hostname, Hanshis-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.12.200.35 instead (on interface en0)
24/08/15 16:24:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 16:24:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [3]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [6]:
import os
base_dir = '/Users/hanshitang/Documents/ADS/Proj1/project-1-individual-HanshiTang/data'
data_folders = [
'landing/tlc_data/2022',
'landing/tlc_data/2023',
'landing/tlc_data/2024',
'raw/tlc_data',
'curated/tlc_data/first_clean',
'curated/tlc_data/final_data'
]

for folder in data_folders:
    path = os.path.join(base_dir, folder)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f'Created folder: {path}')
        


In [7]:
import requests
import os

URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"
YEAR = ['2022', '2023']
MONTH = range(1, 13)
output_relative_dir = '/Users/hanshitang/Documents/ADS/Proj1/project-1-individual-HanshiTang/data/landing/tlc_data'

for year in YEAR:
    for month in MONTH:
        print(f'Starting download for {year}-{str(month).zfill(2)}')
        month_str = str(month).zfill(2)
        url = f'{URL_TEMPLATE}{year}-{month_str}.parquet'
        output_dir = f"{output_relative_dir}/{year}/{year}-{month_str}.parquet"
        
        response = requests.get(url, verify=True)
        with open(output_dir, 'wb') as file:
            file.write(response.content)
            
        print(f'Finished download for {year}-{month_str}')


Starting download for 2022-01
Finished download for 2022-01
Starting download for 2022-02
Finished download for 2022-02
Starting download for 2022-03
Finished download for 2022-03
Starting download for 2022-04
Finished download for 2022-04
Starting download for 2022-05
Finished download for 2022-05
Starting download for 2022-06
Finished download for 2022-06
Starting download for 2022-07
Finished download for 2022-07
Starting download for 2022-08
Finished download for 2022-08
Starting download for 2022-09
Finished download for 2022-09
Starting download for 2022-10
Finished download for 2022-10
Starting download for 2022-11
Finished download for 2022-11
Starting download for 2022-12
Finished download for 2022-12
Starting download for 2023-01
Finished download for 2023-01
Starting download for 2023-02
Finished download for 2023-02
Starting download for 2023-03
Finished download for 2023-03
Starting download for 2023-04
Finished download for 2023-04
Starting download for 2023-05
Finished d

In [9]:
import requests
import os

URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"
YEAR = ['2024']
MONTH = range(1, 6)
output_relative_dir = '/Users/hanshitang/Documents/ADS/Proj1/project-1-individual-HanshiTang/data/landing/tlc_data'

for year in YEAR:
    for month in MONTH:
        print(f'Starting download for {year}-{str(month).zfill(2)}')
        month_str = str(month).zfill(2)
        url = f'{URL_TEMPLATE}{year}-{month_str}.parquet'
        output_dir = f"{output_relative_dir}/{year}/{year}-{month_str}.parquet"
        
        response = requests.get(url, verify=True)
        with open(output_dir, 'wb') as file:
            file.write(response.content)
            
        print(f'Finished download for {year}-{month_str}')


Starting download for 2024-01
Finished download for 2024-01
Starting download for 2024-02
Finished download for 2024-02
Starting download for 2024-03
Finished download for 2024-03
Starting download for 2024-04
Finished download for 2024-04
Starting download for 2024-05
Finished download for 2024-05
Starting download for 2024-06
Finished download for 2024-06


In [25]:
# 读取2019年的Parquet文件
df_2019 = spark.read.parquet('../data/landing/tlc_data/2019/*.parquet')

In [26]:
path = "../data/landing/tlc_data"
df_2019_1 = spark.read.parquet(path + "/2019/2019-01.parquet")

In [27]:
# 显示数据 
df_2019_1.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|            1.0|          1.5|       1.0|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.6

In [28]:
df_2019_1.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)

